<h1><center>Laboratorio 9: Optimización de modelos 💯</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos - Primavera 2024</strong></center>

### **Cuerpo Docente:**

- Profesores: Ignacio Meza, Sebastián Tinoco
- Auxiliar: Eduardo Moya
- Ayudantes: Nicolás Ojeda, Melanie Peña, Valentina Rojas

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Javiera Donoso
- Nombre de alumno 2: Belén Órdenes


### **Link de repositorio de GitHub:** [Repositorio](https://github.com/javieradonoso/Repositorio-MDS7202)

### Temas a tratar

- Predicción de demanda usando `xgboost`
- Búsqueda del modelo óptimo de clasificación usando `optuna`
- Uso de pipelines.

### Reglas:

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias.
- Pueden usar cualquer matrial del curso que estimen conveniente.
- Código que no se pueda ejecutar, no será revisado.

### Objetivos principales del laboratorio

- Optimizar modelos usando `optuna`
- Recurrir a técnicas de *prunning*
- Forzar el aprendizaje de relaciones entre variables mediante *constraints*
- Fijar un pipeline con un modelo base que luego se irá optimizando.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

# Importamos librerias útiles

In [186]:
# !pip3 install xgboost

# El emprendimiento de Fiu

Tras liderar de manera exitosa la implementación de un proyecto de ciencia de datos para caracterizar los datos generados en Santiago 2023, el misterioso corpóreo **Fiu** se anima y decide levantar su propio negocio de consultoría en machine learning. Tras varias e intensas negociaciones, Fiu logra encontrar su *primera chamba*: predecir la demanda (cantidad de venta) de una famosa productora de bebidas de calibre mundial. Al ver el gran potencial y talento que usted ha demostrado en el campo de la ciencia de datos, Fiu lo contrata como data scientist para que forme parte de su nuevo emprendimiento.

Para este laboratorio deben trabajar con los datos `sales.csv` subidos a u-cursos, el cual contiene una muestra de ventas de la empresa para diferentes productos en un determinado tiempo.

Para comenzar, cargue el dataset señalado y visualice a través de un `.head` los atributos que posee el dataset.

<i><p align="center">Fiu siendo felicitado por su excelente desempeño en el proyecto de caracterización de datos</p></i>
<p align="center">
  <img src="https://media-front.elmostrador.cl/2023/09/A_UNO_1506411_2440e.jpg">
</p>

In [187]:
import pandas as pd
import numpy as np
from datetime import datetime

df = df = pd.read_csv("/Users/belenordenes/Desktop/LABORATORIO/Lab9/sales.csv")
# 'date' en el formato dd/mm/yy
df['date'] = pd.to_datetime(df['date'], format='%d/%m/%y')

df.head()

,id,date,city,lat,long,pop,shop,brand,container,capacity,price,quantity
0,0,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,glass,500ml,0.96,13280
1,1,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,plastic,1.5lt,2.86,6727
2,2,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,can,330ml,0.87,9848
3,3,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,adult-cola,glass,500ml,1.00,20050
4,4,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,adult-cola,can,330ml,0.39,25696


## 1 Generando un Baseline (5 puntos)

<p align="center">
  <img src="https://media.tenor.com/O-lan6TkadUAAAAC/what-i-wnna-do-after-a-baseline.gif">
</p>

Antes de entrenar un algoritmo, usted recuerda los apuntes de su magíster en ciencia de datos y recuerda que debe seguir una serie de *buenas prácticas* para entrenar correcta y debidamente su modelo. Después de un par de vueltas, llega a las siguientes tareas:

1. Separe los datos en conjuntos de train (70%), validation (20%) y test (10%). Fije una semilla para controlar la aleatoriedad. [0.5 puntos]
2. Implemente un `FunctionTransformer` para extraer el día, mes y año de la variable `date`. Guarde estas variables en el formato categorical de pandas. [1 punto]
3. Implemente un `ColumnTransformer` para procesar de manera adecuada los datos numéricos y categóricos. Use `OneHotEncoder` para las variables categóricas. `Nota:` Utilice el método `.set_output(transform='pandas')` para obtener un DataFrame como salida del `ColumnTransformer` [1 punto]
4. Guarde los pasos anteriores en un `Pipeline`, dejando como último paso el regresor `DummyRegressor` para generar predicciones en base a promedios. [0.5 punto]
5. Entrene el pipeline anterior y reporte la métrica `mean_absolute_error` sobre los datos de validación. ¿Cómo se interpreta esta métrica para el contexto del negocio? [0.5 puntos]
6. Finalmente, vuelva a entrenar el `Pipeline` pero esta vez usando `XGBRegressor` como modelo **utilizando los parámetros por default**. ¿Cómo cambia el MAE al implementar este algoritmo? ¿Es mejor o peor que el `DummyRegressor`? [1 punto]
7. Guarde ambos modelos en un archivo .pkl (uno cada uno) [0.5 puntos]

1. Separar conjuntos de train, validation y test

In [188]:
from sklearn import set_config
set_config(transform_output="pandas")
from sklearn.model_selection import train_test_split

# Fijar semilla para la aleatoriedad
random_seed = 42

# Dividir el dataset en train (70%), y el resto (30%) para validation y test
train_data, temp_data = train_test_split(df, test_size=0.3, random_state=random_seed)

# Luego dividir el resto (temp_data) en validation (20%) y test (10%)
val_data, test_data = train_test_split(temp_data, test_size=0.333, random_state=random_seed)

# Verificar la cantidad de datos en cada subconjunto
print(f'Tamaño del conjunto de entrenamiento: {len(train_data)}')
print(f'Tamaño del conjunto de validación: {len(val_data)}')
print(f'Tamaño del conjunto de prueba: {len(test_data)}')

Tamaño del conjunto de entrenamiento: 5219
Tamaño del conjunto de validación: 1492
Tamaño del conjunto de prueba: 745


2. Implementar FunctionTransformer

In [189]:
from sklearn.preprocessing import FunctionTransformer

# Definir función para extraer día, mes y año

def extract_date_features(df):
    df = df.copy()
    df['day'] = df['date'].dt.day
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year
    return df

# Crear el transformador
date_transformer = FunctionTransformer(extract_date_features)

# Aplicar el transformador
train_data = date_transformer.fit_transform(train_data)
val_data = date_transformer.transform(val_data)
test_data = date_transformer.transform(test_data)

# Convertir las nuevas columnas a categóricas
for col in ['day', 'month', 'year']:
    train_data[col] = train_data[col].astype('category')
    val_data[col] = val_data[col].astype('category')
    test_data[col] = test_data[col].astype('category')

3. Implemente un `ColumnTransformer` para procesar de manera adecuada los datos numéricos y categóricos. Use `OneHotEncoder` para las variables categóricas. 

In [190]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

# Listar las columnas numéricas y categóricas
numeric_features = ['lat', 'long', 'pop', 'price']
categorical_features = ['city', 'shop', 'brand', 'container', 'capacity', 'day', 'month', 'year']

# Crear el ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', SimpleImputer(strategy='mean'), numeric_features),  # Imputar valores faltantes numéricos si es necesario
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features)  # Evitar salida esparsa
    ]
)

# Establecer que el transformador devuelva un DataFrame
preprocessor.set_output(transform='pandas')

ColumnTransformer(transformers=[('num', SimpleImputer(),
                                 ['lat', 'long', 'pop', 'price']),
                                ('cat',
                                 OneHotEncoder(handle_unknown='ignore',
                                               sparse_output=False),
                                 ['city', 'shop', 'brand', 'container',
                                  'capacity', 'day', 'month', 'year'])])

4. Guarde los pasos anteriores en un `Pipeline`, dejando como último paso el regresor `DummyRegressor` para generar predicciones en base a promedios.

In [191]:
from sklearn.pipeline import Pipeline
from sklearn.dummy import DummyRegressor

# Crear el pipeline
dummy_pipeline = Pipeline(steps=[
    ('date_transformer', date_transformer),  # Primero transformamos las fechas
    ('preprocessor', preprocessor),  # Luego aplicamos el preprocesador de columnas
    ('regressor', DummyRegressor(strategy='mean'))  # Finalmente el regresor dummy
])

5. Entrene el pipeline anterior y reporte la métrica `mean_absolute_error` sobre los datos de validación.

In [192]:
# Entrenar el pipeline
dummy_pipeline.fit(train_data, train_data['quantity'])

from sklearn.metrics import mean_absolute_error
# Hacer predicciones y calcular el MAE
val_predictions = dummy_pipeline.predict(val_data)
mae_dummy = mean_absolute_error(val_data['quantity'], val_predictions)
print(f'Mean Absolute Error del DummyRegressor: {mae_dummy}')

Mean Absolute Error del DummyRegressor: 13302.64227145915


*¿Cómo se interpreta esta métrica para el contexto del negocio?*

El MAE de 13302.64 indica que, en promedio, el modelo está fallando por esa cantidad en la predicción del número de productos vendidos. Esto sugiere que el modelo DummyRegressor, que solo predice el promedio de las ventas, no es útil para tomar decisiones precisas sobre inventario o planificación de ventas, ya que el margen de error es demasiado alto.

6. Finalmente, vuelva a entrenar el `Pipeline` pero esta vez usando `XGBRegressor` como modelo **utilizando los parámetros por default**. 

In [193]:
from xgboost import XGBRegressor
# Crear el pipeline con XGBRegressor
xgb_regressor = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor(random_state=random_seed))
])

# Entrenar el pipeline
xgb_regressor.fit(train_data.drop(columns=['quantity', 'date']), train_data['quantity'])

# Hacer predicciones y calcular el MAE
val_predictions_xgb_regressor = xgb_regressor.predict(val_data.drop(columns=['quantity', 'date']))
mae_xgb_regressor = mean_absolute_error(val_data['quantity'], val_predictions_xgb_regressor)
print(f'Mean Absolute Error del XGBRegressor: {mae_xgb_regressor}')

Mean Absolute Error del XGBRegressor: 2432.184800217043


*¿Cómo cambia el MAE al implementar este algoritmo? ¿Es mejor o peor que el `DummyRegressor`?*

El nuevo MAE de 2432.18 obtenido con el XGBRegressor es significativamente menor que el MAE de 13302.64 obtenido con el DummyRegressor. Esto indica que el XGBRegressor es mucho mejor en capturar patrones en los datos y en hacer predicciones más precisas.

7. Guarde ambos modelos en un archivo .pkl (uno cada uno)

In [194]:
import joblib
# Guardar el DummyRegressor
joblib.dump(dummy_pipeline, 'dummy_model.pkl')

# Guardar el XGBRegressor
joblib.dump(xgb_regressor, 'xgb_regressor_model.pkl')

['xgb_regressor_model.pkl']

## 2. Forzando relaciones entre parámetros con XGBoost (10 puntos)

<p align="center">
  <img src="https://64.media.tumblr.com/14cc45f9610a6ee341a45fd0d68f4dde/20d11b36022bca7b-bf/s640x960/67ab1db12ff73a530f649ac455c000945d99c0d6.gif">
</p>

Un colega aficionado a la economía le *sopla* que la demanda guarda una relación inversa con el precio del producto. Motivado para impresionar al querido corpóreo, se propone hacer uso de esta información para mejorar su modelo realizando las siguientes tareas:

1. Vuelva a entrenar el `Pipeline` con `XGBRegressor`, pero esta vez forzando una relación monótona negativa entre el precio y la cantidad. Para aplicar esta restricción apóyese en la siguiente <a href = https://xgboost.readthedocs.io/en/stable/tutorials/monotonic.html>documentación</a>. [6 puntos]

>Hint 1: Para implementar el constraint se le sugiere hacerlo especificando el nombre de la variable. De ser así, probablemente le sea útil **mantener el formato de pandas** antes del step de entrenamiento.

>Hint 2: Puede obtener el nombre de las columnas en el paso anterior al modelo regresor mediante el método `.get_feature_names_out()`

2. Luego, vuelva a reportar el `MAE` sobre el conjunto de validación. [1 puntos]

3. ¿Cómo cambia el error al incluir esta relación? ¿Tenía razón su amigo? [2 puntos]

4. Guarde su modelo en un archivo .pkl [1 punto]

1. Vuelva a entrenar el `Pipeline` con `XGBRegressor`, pero esta vez forzando una relación monótona negativa entre el precio y la cantidad.

In [195]:
# Ajustar el preprocessor en los datos de entrenamiento y obtener los nombres de las características transformadas
preprocessor.fit(train_data.drop(columns=['quantity', 'date']))
feature_names = preprocessor.get_feature_names_out()
print("Nombres de las características transformadas:", feature_names)

# Definir las restricciones de monotonicidad basadas en los nombres de las características transformadas
monotone_constraints = {name: 0 for name in feature_names}
monotone_constraints['num__price'] = -1  # Monotonicidad negativa para la característica 'price'

print("Restricciones de monotonicidad:", monotone_constraints)

# Crear el pipeline con XGBRegressor y las restricciones de monotonicidad
xgb_monotonic_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor(
        random_state=random_seed,
        monotone_constraints=tuple(monotone_constraints.values())  # Convertir a tuple
    ))
])

# Entrenar el pipeline
xgb_monotonic_pipeline.fit(train_data.drop(columns=['quantity', 'date']), train_data['quantity'])


Nombres de las características transformadas: ['num__lat' 'num__long' 'num__pop' 'num__price' 'cat__city_Athens'
 'cat__city_Irakleion' 'cat__city_Larisa' 'cat__city_Patra'
 'cat__city_Thessaloniki' 'cat__shop_shop_1' 'cat__shop_shop_2'
 'cat__shop_shop_3' 'cat__shop_shop_4' 'cat__shop_shop_5'
 'cat__shop_shop_6' 'cat__brand_adult-cola' 'cat__brand_gazoza'
 'cat__brand_kinder-cola' 'cat__brand_lemon-boost'
 'cat__brand_orange-power' 'cat__container_can' 'cat__container_glass'
 'cat__container_plastic' 'cat__capacity_1.5lt' 'cat__capacity_330ml'
 'cat__capacity_500ml' 'cat__day_28' 'cat__day_29' 'cat__day_30'
 'cat__day_31' 'cat__month_1' 'cat__month_2' 'cat__month_3' 'cat__month_4'
 'cat__month_5' 'cat__month_6' 'cat__month_7' 'cat__month_8'
 'cat__month_9' 'cat__month_10' 'cat__month_11' 'cat__month_12'
 'cat__year_2012' 'cat__year_2013' 'cat__year_2014' 'cat__year_2015'
 'cat__year_2016' 'cat__year_2017' 'cat__year_2018']
Restricciones de monotonicidad: {'num__lat': 0, 'num__long': 0

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', SimpleImputer(),
                                                  ['lat', 'long', 'pop',
                                                   'price']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['city', 'shop', 'brand',
                                                   'container', 'capacity',
                                                   'day', 'month', 'year'])])),
                ('regressor',
                 XGBRegressor(base_score=None, booster=None, callbacks=None,
                              colsample_bylevel=None, co...
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=None, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=(0, 0, 0, -1, 0, 0, 0, 0, 0,
                                                    0, 0, 0, 0, 0, 0, 0, 0, 0,
                                                    0, 0, 0, 0, 0, 0, 0, 0, 0,
                                                    0, 0, 0, ...),
                              multi_strategy=None, n_estimators=None,
                              n_jobs=None, num_parallel_tree=None,
                              random_state=42, ...))])

2. Vuelva a reportar el `MAE` sobre el conjunto de validación.

In [196]:
# Hacer predicciones y calcular el MAE
val_predictions_xgb_monotonic = xgb_monotonic_pipeline.predict(val_data.drop(columns=['quantity', 'date']))

mae_xgb_monotonic = mean_absolute_error(val_data['quantity'], val_predictions_xgb_monotonic)
print(f'Mean Absolute Error del XGBRegressor con restricción monótona: {mae_xgb_monotonic}')

Mean Absolute Error del XGBRegressor con restricción monótona: 2485.9071259460243


3. ¿Cómo cambia el error al incluir esta relación? ¿Tenía razón su amigo?

El MAE anterior nos dio un valor de 2432.18 lo que es menor al MAE actual de un valor de 2485.90, por lo que, la información que nos dieron empeoro el modelo, aunque la diferencia es poca.

4. Guarde su modelo en un archivo .pkl

In [197]:
# Guardamos el XGBRegressor con la restriccion monotónica
joblib.dump(xgb_monotonic_pipeline, 'monotonic_xgb_model.pkl')

['monotonic_xgb_model.pkl']

## 1.3 Optimización de Hiperparámetros con Optuna (20 puntos)

<p align="center">
  <img src="https://media.tenor.com/fmNdyGN4z5kAAAAi/hacking-lucy.gif">
</p>

Luego de presentarle sus resultados, Fiu le pregunta si es posible mejorar *aun más* su modelo. En particular, le comenta de la optimización de hiperparámetros con metodologías bayesianas a través del paquete `optuna`. Como usted es un aficionado al entrenamiento de modelos de ML, se propone implementar la descabellada idea de su jefe.

A partir de la mejor configuración obtenida en la sección anterior, utilice `optuna` para optimizar sus hiperparámetros. En particular, se pide que su optimización considere lo siguiente:

- Fijar una semilla en las instancias necesarias para garantizar la reproducibilidad de resultados
- Utilice `TPESampler` como método de muestreo
- De `XGBRegressor`, optimice los siguientes hiperparámetros:
    - `learning_rate` buscando valores flotantes en el rango (0.001, 0.1)
    - `n_estimators` buscando valores enteros en el rango (50, 1000)
    - `max_depth` buscando valores enteros en el rango (3, 10)
    - `max_leaves` buscando valores enteros en el rango (0, 100)
    - `min_child_weight` buscando valores enteros en el rango (1, 5)
    - `reg_alpha` buscando valores flotantes en el rango (0, 1)
    - `reg_lambda` buscando valores flotantes en el rango (0, 1)
- De `OneHotEncoder`, optimice el hiperparámetro `min_frequency` buscando el mejor valor flotante en el rango (0.0, 1.0)

Para ello se pide los siguientes pasos:
1. Implemente una función `objective()` que permita minimizar el `MAE` en el conjunto de validación. Use el método `.set_user_attr()` para almacenar el mejor pipeline entrenado. [10 puntos]
2. Fije el tiempo de entrenamiento a 5 minutos. [1 punto]
3. Optimizar el modelo y reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto? [3 puntos]
4. Explique cada hiperparámetro y su rol en el modelo. ¿Hacen sentido los rangos de optimización indicados? [5 puntos]
5. Guardar su modelo en un archivo .pkl [1 punto]

1. Implemente una función `objective()` que permita minimizar el `MAE` en el conjunto de validación. Use el método `.set_user_attr()` para almacenar el mejor pipeline entrenado.

In [198]:
import optuna
from optuna.samplers import TPESampler
optuna.logging.set_verbosity(optuna.logging.WARNING)

# Definir la función objective para Optuna
def objective(trial):
    # Definir los espacios de búsqueda para los hiperparámetros
    learning_rate = trial.suggest_float('learning_rate', 0.001, 0.1)
    n_estimators = trial.suggest_int('n_estimators', 50, 1000)
    max_depth = trial.suggest_int('max_depth', 3, 10)
    max_leaves = trial.suggest_int('max_leaves', 0, 100)
    min_child_weight = trial.suggest_int('min_child_weight', 1, 5)
    reg_alpha = trial.suggest_float('reg_alpha', 0.0, 1.0)
    reg_lambda = trial.suggest_float('reg_lambda', 0.0, 1.0)
    
    # Hiperparámetro para OneHotEncoder
    min_frequency = trial.suggest_float('min_frequency', 0.0, 1.0)
    
    # Crear el preprocesador con el valor optimizado de OneHotEncoder
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', SimpleImputer(strategy='mean'), numeric_features),
            ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False, min_frequency=min_frequency), categorical_features)
        ]
    )
    preprocessor.set_output(transform='pandas')

    # Definir el modelo con los hiperparámetros seleccionados por Optuna
    model = XGBRegressor(
        learning_rate=learning_rate,
        n_estimators=n_estimators,
        max_depth=max_depth,
        max_leaves=max_leaves,
        min_child_weight=min_child_weight,
        reg_alpha=reg_alpha,
        reg_lambda=reg_lambda,
        random_state=random_seed 
    )

    # Crear el pipeline
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('regressor', model)
    ])

    # Entrenar el pipeline
    pipeline.fit(train_data.drop(columns=['quantity', 'date']), train_data['quantity'])

    # Predecir en el conjunto de validación y calcular MAE
    val_predictions = pipeline.predict(val_data.drop(columns=['quantity', 'date']))
    mae = mean_absolute_error(val_data['quantity'], val_predictions)
    
    # Almacenar el mejor pipeline
    trial.set_user_attr("best_pipeline_optuna", pipeline)
    
    return mae

# Inicializar el sampler de Optuna (TPESampler)
sampler = TPESampler(seed=random_seed)

# Crear el estudio de Optuna
study = optuna.create_study(direction="minimize", sampler=sampler)

2. Fije el tiempo de entrenamiento a 5 minutos.

In [199]:
# Limite de 5 min
time_max = 300

3. Optimizar el modelo y reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto?

In [200]:
# optimizar
study.optimize(objective, timeout=time_max, show_progress_bar=True)

# Obtener los mejores hiperparámetros y su MAE asociado
best_params_optuna = study.best_params
best_mae_optuna = study.best_value

# Reportar los resultados
print(f"Número de trials: {len(study.trials)}")
print(f"Mejor MAE: {best_mae_optuna}")
print(f"Mejores hiperparámetros encontrados: {best_params_optuna}")

   0%|          | 00:00/05:00

Número de trials: 209
Mejor MAE: 1909.0533322904448
Mejores hiperparámetros encontrados: {'learning_rate': 0.08631826148949161, 'n_estimators': 915, 'max_depth': 8, 'max_leaves': 92, 'min_child_weight': 4, 'reg_alpha': 0.309996783957102, 'reg_lambda': 0.10456795579701522, 'min_frequency': 0.049850359862952894}


*¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto?*

Observamos que el mejor MAE se reduce de aproximadamente 2.400 a 1.909. Notamos que esto se debe a el uso de Optuna permitio encontrar una combinación de hiperparámetros que optimiza el modelo, resultando en un MAE menor y un rendimiento general superior en comparación con el modelo anterior.

5. Guardar su modelo en un archivo .pkl

In [201]:
# Guardar el mejor pipeline encontrado por Optuna en un archivo .pkl
best_pipeline_optuna = study.best_trial.user_attrs['best_pipeline_optuna']

# Guardar el pipeline en un archivo .pkl
joblib.dump(best_pipeline_optuna, 'optuna_model.pkl')

['optuna_model.pkl']

## 4. Optimización de Hiperparámetros con Optuna y Prunners (17 puntos)

<p align="center">
  <img src="https://i.pinimg.com/originals/90/16/f9/9016f919c2259f3d0e8fe465049638a7.gif">
</p>

Después de optimizar el rendimiento de su modelo varias veces, Fiu le pregunta si no es posible optimizar el entrenamiento del modelo en sí mismo. Después de leer un par de post de personas de dudosa reputación en la *deepweb*, usted llega a la conclusión que puede cumplir este objetivo mediante la implementación de **Prunning**.

Vuelva a optimizar los mismos hiperparámetros que la sección pasada, pero esta vez utilizando **Prunning** en la optimización. En particular, usted debe:

1. Responder: ¿Qué es prunning? ¿De qué forma debería impactar en el entrenamiento? [2 puntos]
2. Redefinir la función `objective()` utilizando `optuna.integration.XGBoostPruningCallback` como método de **Prunning** [10 puntos]
3. Fijar nuevamente el tiempo de entrenamiento a 5 minutos [1 punto]
4. Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto? [3 puntos]
5. Guardar su modelo en un archivo .pkl [1 punto]

Nota: Si quieren silenciar los prints obtenidos en el prunning, pueden hacerlo mediante el siguiente comando:

```
optuna.logging.set_verbosity(optuna.logging.WARNING)
```

De implementar la opción anterior, pueden especificar `show_progress_bar = True` en el método `optimize` para *más sabor*.

Hint: Si quieren especificar parámetros del método .fit() del modelo a través del pipeline, pueden hacerlo por medio de la siguiente sintaxis: `pipeline.fit(stepmodelo__parametro = valor)`

Hint2: Este <a href = https://stackoverflow.com/questions/40329576/sklearn-pass-fit-parameters-to-xgboost-in-pipeline>enlace</a> les puede ser de ayuda en su implementación

1. *¿Qué es prunning? ¿De qué forma debería impactar en el entrenamiento?*

Pruning es una técnica utilizada durante el entrenamiento de modelos de machine learning para detener prematuramente la exploración de ciertas ramas del espacio de búsqueda de hiperparámetros si se determina que no conducen a una mejora significativa en el desempeño del modelo. Esto se logra mediante la evaluación periódica del desempeño del modelo en un conjunto de validación durante el proceso de optimización y la interrupción temprana de las iteraciones que no muestran mejoras significativas.

2. Redefinir la función `objective()` utilizando `optuna.integration.XGBoostPruningCallback` como método de **Prunning**

In [202]:
### FUNCIONA

import optuna
from optuna.integration import XGBoostPruningCallback
import xgboost as xgb  
from optuna.samplers import TPESampler
optuna.logging.set_verbosity(optuna.logging.WARNING)

# Definir la función objective para Optuna
def objective(trial):
    # Definir los espacios de búsqueda para los hiperparámetros
    learning_rate = trial.suggest_float('learning_rate', 0.001, 0.1)
    n_estimators = trial.suggest_int('n_estimators', 50, 1000)
    max_depth = trial.suggest_int('max_depth', 3, 10)
    max_leaves = trial.suggest_int('max_leaves', 0, 100)
    min_child_weight = trial.suggest_int('min_child_weight', 1, 5)
    reg_alpha = trial.suggest_float('reg_alpha', 0.0, 1.0)
    reg_lambda = trial.suggest_float('reg_lambda', 0.0, 1.0)
    
    # Hiperparámetro para OneHotEncoder
    min_frequency = trial.suggest_float('min_frequency', 0.0, 1.0)
    
    # Crear el preprocesador con el valor optimizado de OneHotEncoder
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', SimpleImputer(strategy='mean'), numeric_features),
            ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False, min_frequency=min_frequency), categorical_features)
        ]
    )
    preprocessor.set_output(transform='pandas')

    # Transformar los datos antes de entrenar el modelo
    X_train_processed = preprocessor.fit_transform(train_data.drop(columns=['quantity', 'date']))
    X_val_processed = preprocessor.transform(val_data.drop(columns=['quantity', 'date']))
    
    # Preparar los conjuntos de datos para XGBoost
    dtrain = xgb.DMatrix(X_train_processed, label=train_data['quantity'])
    dval = xgb.DMatrix(X_val_processed, label=val_data['quantity'])
    
    # Definir el modelo con los hiperparámetros seleccionados por Optuna
    model_params = {
        'objective': 'reg:squarederror',
        'learning_rate': learning_rate,
        'max_depth': max_depth,
        'max_leaves': max_leaves,
        'min_child_weight': min_child_weight,
        'reg_alpha': reg_alpha,
        'reg_lambda': reg_lambda,
        'eval_metric': 'mae',
        'verbosity': 0   # Establecer el nivel de verbosidad en 0 para evitar impresiones
    }

    # Agregar el callback de pruning
    pruning_callback = XGBoostPruningCallback(trial, "validation-mae")

    # Entrenar el modelo utilizando XGBoost y el callback
    evals = [(dtrain, 'train'), (dval, 'validation')]
    model = xgb.train(
        model_params,
        dtrain,
        num_boost_round=n_estimators,
        early_stopping_rounds=10,
        evals=evals,
        callbacks=[pruning_callback],
        verbose_eval=False  # Deshabilitar la salida de cada iteración
    )

    # Almacenar el mejor modelo en la variable trial
    trial.set_user_attr("best_model_pruning", model)

    # Predecir en el conjunto de validación y calcular MAE
    val_predictions = model.predict(dval)
    mae = mean_absolute_error(val_data['quantity'], val_predictions)
    
    return mae

# Inicializar el sampler de Optuna (TPESampler)
sampler = TPESampler(seed=random_seed)

# Crear el estudio de Optuna
study = optuna.create_study(direction="minimize", sampler=sampler)

3. Fijar nuevamente el tiempo de entrenamiento a 5 minutos.

In [203]:
time_max = 300

4. Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto? 

In [204]:
# Optimizar los hiperparámetros con Pruning
study.optimize(objective, timeout=time_max, show_progress_bar=True)

# Obtener los mejores hiperparámetros y su MAE asociado
best_params_prunning = study.best_params
best_mae_prunning = study.best_value

# Reportar los resultados
print(f"Número de trials: {len(study.trials)}")
print(f"Mejor MAE: {best_mae_prunning}")
print(f"Mejores hiperparámetros encontrados: {best_params_prunning}")

   0%|          | 00:00/05:00

Número de trials: 903
Mejor MAE: 1940.5210199841865
Mejores hiperparámetros encontrados: {'learning_rate': 0.09822680337912902, 'n_estimators': 639, 'max_depth': 8, 'max_leaves': 94, 'min_child_weight': 3, 'reg_alpha': 0.724292219422075, 'reg_lambda': 0.9999157185477091, 'min_frequency': 0.05789173606255919}


*¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto?*

Los resultados actuales muestran un mayor número de trials (903 vs. 209), lo que sugiere que el proceso de optimización exploró más combinaciones de hiperparámetros, pero esto no condujo a una mejora significativa en el MAE, ya que los valores de error final son comparables (1940.52 frente a 1910.94). 

El cambio en los resultados puede deberse a la naturaleza del proceso de optimización, que depende de los valores de inicio, el número de ensayos, y cómo se exploran los hiperparámetros. Además, el uso de más ensayos permite explorar mejor el espacio de búsqueda, lo que puede llevar a diferentes combinaciones de hiperparámetros con resultados muy similares en desempeño, como se observa en este caso.

5. Guardar su modelo en un archivo .pkl

In [205]:
# Guardar el modelo en un archivo .pkl
best_model_pruning = study.best_trial.user_attrs["best_model_pruning"]

# Guardar el modelo en un archivo .pkl utilizando joblib
joblib.dump(best_model_pruning, 'pruning_model.pkl')

['pruning_model.pkl']

## 5. Visualizaciones (5 puntos)

<p align="center">
  <img src="https://media.tenor.com/F-LgB1xTebEAAAAd/look-at-this-graph-nickelback.gif">
</p>


Satisfecho con su trabajo, Fiu le pregunta si es posible generar visualizaciones que permitan entender el entrenamiento de su modelo.

A partir del siguiente <a href = https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/005_visualization.html#visualization>enlace</a>, genere las siguientes visualizaciones:

1. Gráfico de historial de optimización [1 punto]
2. Gráfico de coordenadas paralelas [1 punto]
3. Gráfico de importancia de hiperparámetros [1 punto]

Comente sus resultados:

4. ¿Desde qué *trial* se empiezan a observar mejoras notables en sus resultados? [0.5 puntos]
5. ¿Qué tendencias puede observar a partir del gráfico de coordenadas paralelas? [1 punto]
6. ¿Cuáles son los hiperparámetros con mayor importancia para la optimización de su modelo? [0.5 puntos]

1. Gráfico de historial de optimización

In [206]:
from optuna.visualization import plot_optimization_history

# grafico historial
plot_optimization_history(study)

2. Gráfico de coordenadas paralelas

In [207]:
from optuna.visualization import plot_parallel_coordinate

# grafico coordenadas paralelas
plot_parallel_coordinate(study)

3. Gráfico de importancia de hiperparámetros

In [208]:
from optuna.visualization import plot_param_importances

# grafico importancia
plot_param_importances(study)

## 6. Síntesis de resultados (3 puntos)

Finalmente:

1. Genere una tabla resumen del MAE en el conjunto de validación obtenido en los 5 modelos entrenados desde Baseline hasta XGBoost con Constraints, Optuna y Prunning. [1 punto]
2. Compare los resultados de la tabla y responda, ¿qué modelo obtiene el mejor rendimiento? [0.5 puntos]
3. Cargue el mejor modelo, prediga sobre el conjunto de **test** y reporte su MAE. [0.5 puntos]
4. ¿Existen diferencias con respecto a las métricas obtenidas en el conjunto de validación? ¿Porqué puede ocurrir esto? [1 punto]

1. Tabla resumen de MAE

In [211]:
mae_values = {
    'Model': ['Model dummy', 'Model XGBRegressor', 'Model XGB Monotonic', 'Model Optuna', 'Model Pruning' ],  # nombres de los modelos
    'MAE Val': [mae_dummy, mae_xgb_regressor, mae_xgb_monotonic, best_mae_optuna, best_mae_prunning],  # valores de MAE para el conjunto de entrenamiento
}

# Crear el DataFrame resumen
df_mae_summary = pd.DataFrame(mae_values)
df_mae_summary

,Model,MAE Val
0,Model dummy,13302.642271
1,Model XGBRegressor,2432.184800
2,Model XGB Monotonic,2485.907126
3,Model Optuna,1909.053332
4,Model Pruning,1940.521020


2. Comparar resultados

Se observa que el 4to modelo que usa Optuna es aquel que tiene menor MAE de 1909 y le sigue el modelo Pruning con 1940.

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media.tenor.com/8CT1AXElF_cAAAAC/gojo-satoru.gif">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>